In [23]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from collections import Counter
from importlib.machinery import SourceFileLoader
from os.path import join
from torchtext.vocab import GloVe
import seaborn as sns

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import *
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords' ,quiet=True)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

import re

STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup

from google.colab import files, drive
import os

In [24]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
% cd drive/MyDrive

[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


In [26]:
# Read csvs
df_train = pd.read_csv("finance_train.csv")
df_test = pd.read_csv("finance_test.csv")

# Constants
PUNCTUATION = '!#$%&()*,-./:;<=>?@^_`{|}~'
PUNCTUATION_RE = re.compile("[%s]" % PUNCTUATION)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))


# Hyperparameters
MAX_NB_WORDS = None
MAX_SEQUENCE_LENGTH = None
EPOCHS = None
EMBEDDING_DIM = None
BATCH_SIZE = None

# Clean text - returns lowercase text with removed chars and stopwords
def clean_text(text:str):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = text.replace('x', '')
    text = PUNCTUATION_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    return text

# Padding and indexing of unique words
def pad_sequences_train(train, test):
  tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters=PUNCTUATION, lower=True)
  tokenizer.fit_on_texts(train['Sentence'].values)
  word_index = tokenizer.word_index
  X = tokenizer.texts_to_sequences(train['Sentence'].values)
  X_train = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
  Xt = tokenizer.texts_to_sequences(test['Sentence'].values)
  X_test = pad_sequences(Xt, maxlen=MAX_SEQUENCE_LENGTH)
  return X_train, X_test

# Run model
def run_model(xtr, ytr, xt, yt, labelnum, epochs=5, max_sequence_length=256, max_nb_words=1000, embedding_dim=300):
  # Problematic input
  if any(x is None for x in [xtr, ytr, xt, yt, epochs, max_sequence_length, max_nb_words, embedding_dim]):
    print('Replace the None values above with your new values before calling the run_model() function.')
    return None, None, None
  
  # NN
  model = Sequential() # Container
  model.add(Embedding(max_nb_words+1, embedding_dim, mask_zero=True, input_length=max_sequence_length)) # Embedding
  model.add(SpatialDropout1D(0.2)) # Dropout
  model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) # LSTM layer
  model.add(Dense(labelnum, activation='softmax')) # Densely connected layer with softmax activation
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Performance
  print(model.summary())
  history = model.fit(xtr, 
                    ytr, 
                    epochs=epochs, 
                    batch_size=BATCH_SIZE,
                    validation_split=0.2,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
  test_loss, test_accuracy = model.evaluate(xt, yt)
  return model, history, test_accuracy

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#Data Preprocessing


In [27]:
# Get rid of unwanted characters and punctuation to reduce noise
df_train["Sentence"] = df_train["Sentence"].map(clean_text)
df_test["Sentence"] = df_test["Sentence"].map(clean_text)

In [28]:
df_train.head()
df_test.head()

,Sentence,Label
0,third quarter 2010 net sales increased 52 eur ...,2
1,foundries division reports sales increased 97 ...,2
2,financing project come mainly china,1
3,sukhraj dulai 2900 block boni sue court culdes...,1
4,finland leading metals group outokumpu said fo...,2


In [29]:
MAX_SEQUENCE_LENGTH = 256
MAX_NB_WORDS = 1000


In [30]:
# Pad X train and X test, with the model fitted to X train
X_train, X_test = pad_sequences_train(df_train, df_test)
X_train

array([[  0,   0,   0, ..., 347,  91, 161],
       [  0,   0,   0, ..., 285,  93, 484],
       [  0,   0,   0, ...,  39, 185, 654],
       ...,
       [  0,   0,   0, ...,  32,  11,  16],
       [  0,   0,   0, ...,  11,  94,  15],
       [  0,   0,   0, ..., 909,  33, 218]], dtype=int32)

In [31]:
X_test

array([[  0,   0,   0, ...,   4,   1,   2],
       [  0,   0,   0, ...,  32,  11,  31],
       [  0,   0,   0, ..., 786, 534, 270],
       ...,
       [  0,   0,   0, ...,  25, 954, 363],
       [  0,   0,   0, ..., 536, 931,  27],
       [  0,   0,   0, ...,   2,   1,   2]], dtype=int32)

In [32]:
# Get dummies to turn labels into a format where it can be processed by a probabilistic model
Y_train = pd.get_dummies(df_train["Label"]).values
Y_test = pd.get_dummies(df_test["Label"]).values
# Y_train

In [33]:
# Y_test

In [34]:
# Initialize model
label_count = 3
label_map = {0:"negative", 1:"neutral", 2:"positive"}

model = run_model(X_train, Y_train, X_test, Y_test, label_count, 5, 256, 1000, 300)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 256, 300)          300300    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 256, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 461,003
Trainable params: 461,003
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
51/51 [==============================] - 90s 2s/step - loss: 0.8588 - accuracy: 0.6900 - val_loss: 0.6752 - val_accuracy: 0.7451
Epoch 2/5
51/51 [==============================] - 85s 2s/step - loss: 0.4770 - accuracy: 0.8158 - val